A continuación, elaboraré un modelo de regresión múltiple que incluirá únicamente las variables seleccionadas mediante un proceso de selección hacia adelante y eliminación hacia atrás.



*   "acidezFija". La acidez fija del vino, medida en gramos de ácido tartárico por decímetro cúbico.
*   "acidezVolatil". La acidez volátil del vino, medida en gramos de ácido acético por decímetro cúbico.
*   "acidoCitrico". Gramos de táctico cítrico por decímetro cúbico.
*   "azucarResidual". Gramos de tácito cítrico por decímetro cúbico.
*   "cloruros". Gramos de cloruro de sodio por decímetro cúbico.
*   "dioxidoAzufreLibre".Miligramos de dióxido de azufre libre por decímetro cúbico.
*   "densidad". Medida en gramos por centímetro cúbico.
*   "pH". Valor del vino en la escla de pH.
*   "sulfatos". Gramoms de sulfato de potasio por decímetro cúbico.
*   "alcohol". Volúmen percentil de alcohol en el vino.
*   "calidad". Mediana de la calidad otorgada por al menos tres catadores, en escala del 0 (muy malo) al 10 (excelente)













In [34]:
# primero importamos el Archivo Vino Tinto.csv
import pandas as pd
datos = pd.read_csv('A1.4 Vino Tinto.csv')
print(datos.shape)
print(datos.head(5))

(1599, 12)
   acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
0         7.4           0.70          0.00             1.9     0.076   
1         7.8           0.88          0.00             2.6     0.098   
2         7.8           0.76          0.04             2.3     0.092   
3        11.2           0.28          0.56             1.9     0.075   
4         7.4           0.70          0.00             1.9     0.076   

   dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  alcohol  \
0                11.0                34.0    0.9978  3.51      0.56      9.4   
1                25.0                67.0    0.9968  3.20      0.68      9.8   
2                15.0                54.0    0.9970  3.26      0.65      9.8   
3                17.0                60.0    0.9980  3.16      0.58      9.8   
4                11.0                34.0    0.9978  3.51      0.56      9.4   

   calidad  
0        5  
1        5  
2        5  
3        6  
4        5

A continuación, dividiré el DataFrame en dos conjuntos: uno de entrenamiento y otro de prueba, utilizando una proporción de 80/20. El 80% de los datos se destinarán al entrenamiento del modelo, mientras que el 20% restante se utilizará para su evaluación.

In [35]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
train, test = train_test_split(datos, train_size =0.8) # Dividimos a una proporción de 80/20

# Dividir los datos en conjuntos de entrenamiento y prueba
X = datos.drop('calidad', axis=1)
y = datos['calidad']

# Se divide en datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

# Imprimimos las dimensiones de ambos conjuntos y verificamos que sumen la cantidad de datos original
print(" Dimensiones de conjunto de entrenamiento:", train.shape)
print(" Dimensiones de conjunto de prueba:", test.shape)
print(" Suma de dimensiones:", train.shape[0] + test.shape[0])

 Dimensiones de conjunto de entrenamiento: (1279, 12)
 Dimensiones de conjunto de prueba: (320, 12)
 Suma de dimensiones: 1599


Después, utilizare la función SequentialFeatureSelector de la librería mlxtend.feature_selection, la cual  permitirá seleccionar las variables más importantes para construir nuestro modelo.

In [36]:
# Importamos librerias necesarias
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression

# A continuación creamos el modelo de regresión Lineal
model = LinearRegression()

# Despues realizaremos la selección hacia adelante
# Use SFS instead of sf to call SequentialFeatureSelector
sf = SFS(model,k_features=(2,8),forward=True,floating=False,scoring='r2',cv=10)
  # Entre 2 y 8 caracteristicas  #Hacia adelante   #que no sea flotante    #Se usa R cuadrada como métrica

# Se ajusta el modelo
sf = sf.fit(X,y)

# Y se obtienen las caracteristicas que se seleccionaron
sf = sf.k_feature_names_
print("Las caracteristicas de la seleccion (hacia adelante) son: \n",sf)


Las caracteristicas de la seleccion (hacia adelante) son: 
 ('acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol')


A continuación, entrenaré un modelo de regresión lineal utilizando únicamente las características seleccionadas en el paso anterior. Luego, realizaré predicciones sobre el conjunto de datos de prueba y evaluaré la capacidad de predicción mediante la R cuadrada. Esto permitirá medir qué tan bien el modelo ajusta los datos y qué tan precisa es su capacidad de predicción en nuevas observaciones.

In [37]:
# Importamos la función para calcular el R^2
from sklearn.metrics import r2_score

# Depues se entrena el modelo
model.fit(X_train[list(sf)], y_train)

# Y se hacen predicciones sobre el conjunto de prueba
ypred = model.predict(X_test[list(sf)])

# Y por ultimo se calcula la R^2
r2 = r2_score(y_test, ypred)
print("R^2:", r2)

R^2: 0.3713750871303617


A continuación, realizaré el proceso de selección de características hacia atrás, comenzando con las variables seleccionadas previamente por el método de selección hacia adelante. Después, imprimiré en consola los índices de las variables que han sido seleccionadas en este nuevo proceso.

In [38]:
# Igual que en forward pero hacia atras y con menos variables a seleccionar
sb = SFS(model,k_features=(2,5),forward=False,floating=False,scoring='r2',cv=10)

# Se ajusta el proceso de seleccion hacia atrás utilizando las características del de hacia adelante
sb = sb.fit(X_train[list(sf)], y_train)

# Se obtiene los nombres de las caracteristicas seleccionadas
sfb = sb.k_feature_names_
print("Las caracteristicas de la selccion (hacia atras) son: \n",sfb)

# Se entrena el modelo utilizando las caracteristicas que se seleccionaron
model.fit(X_train[list(sfb)], y_train)

# Nuevamente se hacen las predicciones sobre el conjunto de prueba
ypredb = model.predict(X_test[list(sfb)])

# Se calcula el R^2
r2 = r2_score(y_test, ypredb)
print("R^2:", r2)

# Comentario sobre diferencia de R^2 de ambos modelos
print("Al comparar los modelos, el que usa la selección hacia adelante parece tener un mejor desempeño, \n"
      "ya que su R^2 de 0.3058 muestra que solo explica el 30.58% de la variabilidad de lo que estamos tratando de predecir. \n"
      "Aunque ambos modelos tienen un R^2 bajo, lo que indica que no explican mucho de la variabilidad de los datos, \n"
      "el modelo de selección hacia adelante tiene un ajuste ligeramente mejor. Ninguno de los dos modelos es perfecto, \n"
      "pero el de selección hacia adelante parece entender mejor la relación entre las características y lo que queremos predecir.")

Las caracteristicas de la selccion (hacia atras) son: 
 ('acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol')
R^2: 0.36011479702597815
Al comparar los modelos, el que usa la selección hacia adelante parece tener un mejor desempeño, 
ya que su R^2 de 0.3058 muestra que solo explica el 30.58% de la variabilidad de lo que estamos tratando de predecir. 
Aunque ambos modelos tienen un R^2 bajo, lo que indica que no explican mucho de la variabilidad de los datos, 
el modelo de selección hacia adelante tiene un ajuste ligeramente mejor. Ninguno de los dos modelos es perfecto, 
pero el de selección hacia adelante parece entender mejor la relación entre las características y lo que queremos predecir.
